In [1]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Input
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Bidirectional
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import Reshape
from keras.layers import Average
from keras_contrib.layers import CRF
import keras
from sklearn.model_selection import train_test_split
# define documents
import numpy as np
import pandas as pd
import os
from tqdm.auto import tqdm
tqdm.pandas()
import pickle
# import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
## tokenize the text..
from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
def get_tokens(sentence):
#     tokens = nltk.word_tokenize(sentence)  # now using tweet tokenizer
    tokens = tknzr.tokenize(sentence)
    tokens = [token for token in tokens if (token not in stopwords and len(token) > 1)]
    tokens = [get_lemma(token) for token in tokens]
    return (tokens)

In [3]:
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

In [4]:
from sklearn.utils import shuffle
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

In [5]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [6]:
# setup env
import nltk
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/samarthgoal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/samarthgoal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
dir_path = "/home/samarthgoal/language_vision/input/reddit/"

In [8]:
embedding_path = "/home/samarthgoal/embeddings/glove.twitter.27B.100d.txt" ## change

In [9]:
top_dir = "/home/samarthgoal/language_vision"
model_dir = os.path.join(top_dir,"models")

In [10]:
df_rage = pd.read_csv(os.path.join(dir_path,'processed_rage.csv'))
df_happy =  pd.read_csv(os.path.join(dir_path,'processed_happy.csv'))
df_gore =  pd.read_csv(os.path.join(dir_path,'processed_gore.csv'))
df_creepy =  pd.read_csv(os.path.join(dir_path,'processed_creepy.csv'))

In [11]:
# create a random balances dataset of all of the categories
length = np.min([len(df_rage),len(df_happy),len(df_creepy),len(df_gore)])

In [12]:
## combing all of the dataset
df_final = pd.concat([df_rage[:length], df_happy[:length], df_gore[:length], df_creepy[:length]], ignore_index=True)

### img_preprocessing

In [13]:
import urllib.request
import os
import pandas as pd
from io import BytesIO
import numpy as np
from sklearn.manifold import TSNE
# import matplotlib.pyplot as plt
import pickle
from keras.layers import Lambda, Input
from keras.models import Model
from keras.backend import tf as ktf
import urllib.request
from requests.exceptions import ConnectionError
tqdm.pandas()

In [14]:
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input as preprocess_input_incept
from keras.layers import MaxPooling1D
from keras.layers import Maximum
from keras.layers import Concatenate
from keras.layers import Dense
from keras.layers import Flatten
import numpy as np
from keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
import urllib3

In [15]:
## trying with img1
# @ param: img url @ return feature vector 
from PIL import Image
import requests

In [16]:
def get_img_data(url):
    try : 
        response = requests.get(url)
        img_data = 0
        if (response.status_code == 200):
            try:
                if (url.endswith('.jpg') or url.endswith('.jpeg') or url.endswith('.png')):
                    with urllib.request.urlopen(url) as url1:
                        with open('temp.jpg', 'wb') as f:
                            f.write(url1.read())
                    fh = open('temp.jpg')
                    if fh:
                        img = image.load_img('temp.jpg', target_size=(224, 224)) # 224*224
                        os.remove('temp.jpg')
                        img_data = image.img_to_array(img)
                    return img_data
            except RuntimeError as e:
                return 0  # except runtime error fro file that doesn't exits
        return img_data
    except IOError:
        return 0
    except FileNotFoundError as e:
        return 0
    except ConnectionError as e:
        return 0
    except urllib.error.HTTPError as e:
        return 0

In [17]:
## takes a lot of time <40 min> .. Dont run!! pickle_load
# img_data = np.array(list(df_final['url'].progress_apply(get_img_data))) ## @TODO uncomment

In [18]:
## load the image data
img_data = pickle.load(open(os.path.join(model_dir,"img_data.pkl"),"rb"))

In [19]:
## filter the data that doesn't have images
indices = ([idx for idx,ele in enumerate(img_data) if np.size(ele) == 1])
print(len(indices))
df_final = df_final.loc[~df_final.index.isin(indices)]

987


In [20]:
len(df_final)

1473

In [21]:
# create a random balances dataset of all of the categories
Y_new = df_final['subreddit']

In [22]:
# encode the output
Y_new = le.fit_transform(Y_new) 

In [23]:
print(len(Y_new))
print(len(df_final))

1473
1473


In [ ]:
## train test slplit
ids = list(df_final.id)
train_ids,test_ids,Y_train,Y_test = train_test_split(ids,Y_new,test_size=0.20,random_state=6)

In [ ]:
print(len(train_ids))
print(len(test_ids))

1178
295


In [ ]:
## get the training imgae data
indices = df_final.index[df_final.id.isin(train_ids) == True].tolist()

In [ ]:
img_data_train = np.array([img for idx,img in enumerate(img_data) if idx in indices])

In [ ]:
## preprocess img_data
img_data_train = preprocess_input(img_data_train)

In [ ]:
(img_data_train).shape

(1178, 224, 224, 3)

### text preprocessing

In [ ]:
token_list = (df_final['title'].apply(get_tokens))

In [ ]:
## join the filtered tokens back again to the keras tokenizer which would give vocalb words etc
from keras.preprocessing.text import Tokenizer
sentences = [' '.join(tokens) for tokens in token_list]
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(token_list)
vocab_size = len(t.word_index) + 1
vocab_size

3687

In [ ]:
len(token_list)

1473

In [ ]:
max_len = 0
for idx,tokens in enumerate(token_list):
    if len(tokens) > max_len:
        max_len = len(tokens)
        index = idx
print(max_len)

34


In [ ]:
df_train = df_final.loc[df_final.id.isin(train_ids)]
train_tokens = df_train['title'].apply(get_tokens)

In [ ]:
len(train_tokens)

1178

In [ ]:
train_sentences = [' '.join(tokens) for tokens in train_tokens]

In [ ]:
vocab_size = vocab_size
# integer encode the documents
encoded_docs = t.texts_to_sequences(train_sentences)
# pad documents to a max length of 4 words
max_length = max_len
lng_data_train = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [ ]:
# create the word2vec dict from the dictionary
def get_word2vec(file_path):
    file = open(embedding_path, "r")
    if (file):
        word2vec = dict()
        split = file.read().splitlines()
        for line in split:
            key = line.split(' ',1)[0] # the first word is the key
            value = np.array([float(val) for val in line.split(' ')[1:]])
            word2vec[key] = value
        return (word2vec)
    else:
        print("invalid fiel path")

In [ ]:
## dont run it again!!
# w2v = get_word2vec(embedding_path)

In [ ]:
## load the pre-converted w2v file from models dir
w2v = pickle.load(open(os.path.join(model_dir,"w2v.pkl"),"rb"))

In [ ]:
# get the embedding matrix from the embedding layer
from numpy import zeros
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
	embedding_vector = w2v.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

### final model

In [ ]:
## main model

## text model
input1 = Input(shape=(max_len,))
model =  Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length,trainable=False)(input1)
model =  Bidirectional( LSTM(units=100,return_sequences=True,dropout=0.25),merge_mode="mul")(model)
model = TimeDistributed( Dense(100, activation="relu"))(model)
model = Flatten()(model)
lng_vec = Dense(100, activation = 'relu')(model)

## fusing the embedding from fasttext and glove

## img_model
model_img = VGG16(weights='imagenet', include_top=True)
input2 = model_img.input
img_vec_ = model_img.get_layer('fc2').output
img_vec = Dense(100)(img_vec_)

# hidden =  Average()([lng_vec,img_vec])
hidden = Concatenate()([lng_vec,img_vec])

out = Dense(100,activation= 'relu')(hidden)
out = Dense(4,activation='softmax')(out)

# model = Model(input=input2,output=img_vec)
# out =  (Dense(4, activation = 'softmax'))(model)
# model = Model(input1,l_vec)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
model = Model(inputs=[input1, input2], outputs=out)

In [ ]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 224, 224, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 112, 112, 64) 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [ ]:
img_data_train.shape

(1178, 224, 224, 3)

In [ ]:
lng_data_train.shape

(1178, 34)

In [ ]:
hist = model.fit([lng_data_train,img_data_train],Y_train,validation_split=0.2,nb_epoch = 5, verbose = 1)

Instructions for updating:
Use tf.cast instead.


/home/samarthgoal/.local/lib/python3.5/site-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 942 samples, validate on 236 samples
Epoch 1/5


## prediction

In [ ]:
def get_X(ids,df_final):
    indices = df_final.index[df_final.id.isin(ids) == True].tolist()
    img_data = pickle.load(open(os.path.join(model_dir,"img_data.pkl"),"rb"))
    img_data = np.array([img for idx,img in enumerate(img_data) if idx in indices])
    img_data = preprocess_input(img_data)
    
    ## lng_data
    df_sample = df_final.loc[df_final.id.isin(ids)]
    tokens = df_sample['title'].apply(get_tokens)
    sentences = [' '.join(tokens) for tokens in tokens]
    encoded_docs = t.texts_to_sequences(sentences)
    lng_data = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
    return ([lng_data,img_data])

In [ ]:
X_test = get_X(test_ids,df_final)

In [ ]:
print(X_test[0].shape)
print(X_test[1].shape)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
Y_pred = model.predict(X_test,verbose=1)
y_pred = np.array([np.argmax(pred) for pred in Y_pred])
print('  Classification Report:\n',classification_report(Y_test,y_pred),'\n')

## dump the w2v vec file

In [ ]:
import pickle
with open(os.path.join(model_dir,"w2v.pkl"),"wb") as f:
    pickle.dump(w2v,f)

In [ ]:
## dump the image array
with open(os.path.join(model_dir,"img_data.pkl"),"wb") as f:
    pickle.dump(img_data,f)

In [ ]:
## dump the model
# serialize model to JSON
model_json = model.to_json()
with open(os.path.join(model_dir,"model.json"), "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(os.path.join(model_dir,"model.h5"))
print("Saved model to disk")